# Working with YAML files
- YAML (“YAML Ain’t Markup Language”) focuses on human readability. Indentation replaces braces and brackets, comments are allowed, and quoting is usually optional.
- DevOps tooling (Kubernetes, Ansible, GitHub Actions, many app configs) standardizes on YAML for its clarity and brevity.
- JSON is excellent for machine-to-machine communication, but its strict syntax (no comments, heavy quoting) can feel verbose to humans maintaining config files.
- Python’s standard library lacks YAML support; **PyYAML** is the community-standard package to fill that gap.

## YAML Syntax and Features
- Structure comes from **spaces for indentation**: tabs are discouraged.  
- **Mappings** use `key: value`; **sequences** use a leading hyphen (`-`) plus a space.
- Scalars include strings, numbers, booleans (`true / false`, `yes / no`), and `null`.  
- Comments begin with `#`.
- Multi-line scalars can be literal (`|`) or folded (`>`).  
- **Anchors (&) and aliases (*)** avoid repetition by re-using defined blocks.  
- YAML is a superset of JSON: most valid JSON documents are also valid YAML.

In [2]:
import yaml, json

snippet = """
staging:
  name: user-api
  port: 8080
  enabled: true
  tags:
    - api
    - user
    - internal
  replicas: 2
production:
  name: user-api
  port: 8080
  enabled: true
  tags:
    - api
    - user
    - internal
  replicas: 4
"""

parsed = yaml.safe_load(snippet)
print("Parsed YAML:", parsed)

Parsed YAML: {'staging': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal'], 'replicas': 2}, 'production': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal'], 'replicas': 4}}


In [5]:
# To avoid redundancy, use the following:

import yaml, json

snippet = """
service: &svc
  name: user-api
  port: 8080
  enabled: true
  tags:
    - api
    - user
    - internal
staging: 
    <<: *svc
    replicas: 2
production:
    <<: *svc
    replicas: 4
"""    

parsed = yaml.safe_load(snippet)
print("Parsed YAML:", parsed)


multiline_demo = """
literal: |
  Line 1
  Line 2
  Line 3
folded: >
  This is a long string that
  could go out of screen, so
  we will break this up into multiple lines to improve readability.
"""  
print("\nMultiline string demo:", yaml.safe_load(multiline_demo))

Parsed YAML: {'service': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal']}, 'staging': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal'], 'replicas': 2}, 'production': {'name': 'user-api', 'port': 8080, 'enabled': True, 'tags': ['api', 'user', 'internal'], 'replicas': 4}}

Multiline string demo: {'literal': 'Line 1\nLine 2\nLine 3\n', 'folded': 'This is a long string that could go out of screen, so we will break this up into multiple lines to improve readability.\n'}


## Deserializing YAML with `yaml.safe_load`
- Prefer **`yaml.safe_load`** (or passing `Loader=yaml.SafeLoader`) to prevent arbitrary-code execution; avoid `yaml.load` on untrusted data.
- Accepts a string or an open text file handle and returns native Python structures.  
- Wrap calls in `try / except yaml.YAMLError` to catch malformed input.

In [15]:
import yaml 
from pathlib import Path
import os

compose = os.getcwd() + "\\files-regex-data-formats\\compose.yaml"

try:
    with open(compose, mode="r", encoding="utf-8") as file:
        config = yaml.safe_load(file)
        print(f"Compose version:, {config["version"]}")

        for svc, options in config["services"].items():
            print(f"{svc.capitalize()} image\t: {options["image"]}")
except yaml.YAMLError as e:
    print(f"Error parsing YAML: {e}")

    
    

Compose version:, 3.8
Web image	: myapp:latest
Redis image	: redis:alpine


## Serializing Python Objects with `yaml.dump`
- Use `yaml.dump(obj, indent=2, default_flow_style=False, sort_keys=False)` for readable block-style output.  
- Set `stream` to an open file handle to write directly; leave it `None` to return a string.

In [17]:
import yaml
from pathlib import Path
import os

python_cfg = {
    "service": {"name": "listener-service", "port": 8080, "workers": 4, "enabled": True},
    "queues": ["default", "high", "low"],
    "retry:policy": None
}

output_path = os.getcwd() + "\\files-regex-data-formats\\listener_config.yaml"

with open(output_path, mode="w", encoding="utf-8") as file:
    yaml.dump(python_cfg, file, sort_keys=False, default_flow_style=False)